In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Install YOLOv5 dependencies
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
%cd yolov5
!pip install -r requirements.txt  # Install necessary packages


Cloning into 'yolov5'...
remote: Enumerating objects: 16965, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 16965 (delta 79), reused 99 (delta 49), pack-reused 16805 (from 1)
Receiving objects: 100% (16965/16965), 15.71 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (11612/11612), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.0/874.0 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


**Partitioning the dateset**

In [ ]:
import os
import shutil
import random

# Set seed for reproducibility
random.seed(42)

# Define the Google Drive path to your dataset
base_dir = '/content/drive/MyDrive/datasets/Natural'
indoor_dir = os.path.join(base_dir, 'Indoor')
outdoor_dir = os.path.join(base_dir, 'Outdoor')

# Define destination directories for the partitioned dataset
output_dir = '/content/drive/MyDrive/datasets/Partitioned'
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Create destination directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Combine all images from Indoor and Outdoor into a single list
image_paths = []

for folder in [indoor_dir, outdoor_dir]:
    for file_name in os.listdir(folder):
        if file_name.lower().endswith('.jpg'):
            image_paths.append(os.path.join(folder, file_name))

# Shuffle the images
random.shuffle(image_paths)

# Define split sizes
train_ratio = 0.7
val_ratio = 0.15

train_size = int(len(image_paths) * train_ratio)
val_size = int(len(image_paths) * val_ratio)

# Partition the dataset
train_images = image_paths[:train_size]
val_images = image_paths[train_size:train_size + val_size]
test_images = image_paths[train_size + val_size:]

# Function to copy images to destination folders
def copy_images(image_list, destination_folder):
    for image_path in image_list:
        shutil.copy(image_path, destination_folder)

# Copy images to respective directories
copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

print("Dataset partitioned successfully!")


**Creating The Yaml File**

In [3]:
# dataset.yaml
path: '/content/drive/MyDrive/FingerTipPartitioned'  # Path to the partitioned dataset folder in Google Drive
train: '/content/drive/MyDrive/FingerTipPartitioned/Images/train' # Training images folder
val: '/content/drive/MyDrive/FingerTipPartitioned/Images/val'  # Validation images folder
test : '/content/drive/MyDrive/FingerTipPartitioned/Images/test'

nc: 1  # Number of classes (e.g., 'fingertip')
names: ['FingerTip']  # List of class names


In [4]:
dataset_yaml = """
path: '/content/drive/MyDrive/FingerTipPartitioned'
train: '/content/drive/MyDrive/FingerTipPartitioned/images/train'
val: '/content/drive/MyDrive/FingerTipPartitioned/images/val'
test : '/content/drive/MyDrive/FingerTipPartitioned/images/test'

nc: 1
names: ['FingerTip']
"""

# Save the YAML file in the correct location
with open('/content/yolov5/data/dataset.yaml', 'w') as file:
    file.write(dataset_yaml)


**Train The Yolo Model**

In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [7]:
!python train.py --img 1280 --batch 16 --epochs 50 --data /content/yolov5/data/dataset.yaml --weights yolov5s.pt --cache


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
2024-09-23 06:33:53.193917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 06:33:53.213523: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 06:33:53.219412: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/

In [8]:
# Define the source and destination paths
source = '/content/yolov5/runs/train/exp/weights/best.pt'  # Path where YOLOv5 saves the best model
destination = '/content/drive/MyDrive/best_fingertip_model.pt'  # Path to save in Google Drive

# Copy the model from Colab to Google Drive
!cp {source} {destination}


In [9]:
# Zip the entire runs directory (which contains all training outputs)
!zip -r /content/yolov5_training.zip /content/yolov5/runs/train/

# Copy the zip file to Google Drive
!cp /content/yolov5_training.zip /content/drive/MyDrive/yolov5_training.zip


  adding: content/yolov5/runs/train/ (stored 0%)
  adding: content/yolov5/runs/train/exp/ (stored 0%)
  adding: content/yolov5/runs/train/exp/val_batch0_labels.jpg (deflated 5%)
  adding: content/yolov5/runs/train/exp/P_curve.png (deflated 24%)
  adding: content/yolov5/runs/train/exp/val_batch1_pred.jpg (deflated 4%)
  adding: content/yolov5/runs/train/exp/F1_curve.png (deflated 31%)
  adding: content/yolov5/runs/train/exp/results.png (deflated 8%)
  adding: content/yolov5/runs/train/exp/hyp.yaml (deflated 45%)
  adding: content/yolov5/runs/train/exp/events.out.tfevents.1727073235.fbb7632cf0be.3481.0 (deflated 43%)
  adding: content/yolov5/runs/train/exp/val_batch1_labels.jpg (deflated 4%)
  adding: content/yolov5/runs/train/exp/train_batch1.jpg (deflated 5%)
  adding: content/yolov5/runs/train/exp/confusion_matrix.png (deflated 40%)
  adding: content/yolov5/runs/train/exp/val_batch2_pred.jpg (deflated 4%)
  adding: content/yolov5/runs/train/exp/PR_curve.png (deflated 29%)
  adding: co

Detection

In [14]:
import os
import subprocess
from pathlib import Path
from PIL import Image

def crop_and_resize(image_path, bbox, output_size=(350, 450)):
    """Crops and resizes the image based on the detected bounding box."""
    img = Image.open(image_path)
    cropped_img = img.crop((bbox[0], bbox[1], bbox[2], bbox[3]))  # Crop using the bounding box
    resized_img = cropped_img.resize(output_size)
    return resized_img

# Define the path to the folder with images
images_folder = r'/content/drive/MyDrive/IIITD Fingerphoto dataset/White/Outdoor'

# Define the output directory for detected images
output_folder = r'/content/drive/MyDrive/IIITD Extracted FingerTip/White/Outdoor'
os.makedirs(output_folder, exist_ok=True)  # Create the output directory if it doesn't exist

# Define the YOLOv5 directory
yolov5_dir = r'/content/yolov5'

# Run YOLOv5 detection using subprocess to call the detect.py script
command = [
    "python", os.path.join(yolov5_dir, "detect.py"),
    "--source", images_folder,
    "--weights", "runs/train/exp/weights/best.pt",
    "--img", "640",
    "--conf-thres", "0.25",
    "--save-txt",
    "--save-conf",
    "--save-crop",  # Enable saving cropped images
    "--project", output_folder,  # Specify the output folder for results
    "--name", "fingertip_detection"  # Name of the folder inside the output directory
]

# Execute the YOLOv5 detection command
subprocess.run(command, check=True)

print(f"Detection completed. Results saved to {output_folder}")

Detection completed. Results saved to /content/drive/MyDrive/IIITD Extracted FingerTip/White/Outdoor
